In [19]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import requests
from datetime import datetime
import xarray as xr
import sys
import netCDF4 as nc
from rasterio.transform import Affine
import rasterio as rio

In [20]:
api_auth = '/home/tyler/.api_request'
url = "https://forecasting.energy.arizona.edu/erebos/series/adjghi"

In [21]:
def grid_maker(central_lon=None, central_lat=None, year=None, month=None, day=None, second_index=None):
    
    latz = []
    lonz = []
    
    for i in range(0,9):
        if 0 <= i < 4:
            dalats_S = central_lat - (0.009 * i) 
            dalons_E = central_lon + (0.009 * i)
            latz.append(dalats_S)
            lonz.append(dalons_E)
        if i == 4:
            latz.append(central_lat)
            lonz.append(central_lon)
        if i >= 5:
            dalats_N = central_lat + (0.009 * i)
            dalons_W = central_lon - (0.009 * i)
            latz.append(dalats_N)
            lonz.append(dalons_W)
    
    ghi_array = []
    
    print("Part 1 is done")
    
    for i in range(len(latz)):
        for j in range(len(lonz)):
            args = {'run_date':'{0}-{1}-{2}'.format(year,month,day), 
                    'lon':'{}'.format(lonz[j]),
                    'lat':'{}'.format(latz[i]),
                    'precipitable_water':'1.00',
                    'aod700':'0.05'}
            with open(api_auth) as f:
                auth_text = f.read()
            auth_tuple = tuple(auth_text.split('\n'))[:2]
            x = requests.get(url, params=args, auth=auth_tuple)
            df = pd.DataFrame(x.json())
            time = np.arange(0,288,1)
            ghi = df['results'].squeeze().values
            ghi_array.append(ghi[second_index])
            
    ghi1 = np.asarray(ghi_array)
    latz = np.asarray(latz)
    lonz = np.asarray(lonz)
     
    filename = 'erebos_grid_{0}-{1}-{2}_{3}.nc'.format(year,month,day,second_index)
    
    ghi1 = ghi1.reshape(9,9)
    
    ds = nc.Dataset(filename, 'w', format='NETCDF4')
    
    time = ds.createDimension('time', None)
    lat = ds.createDimension('lat', 9)
    lon = ds.createDimension('lon', 9)
    
    times = ds.createVariable('time', 'f4', ('time',))
    lats = ds.createVariable('lat', 'f4', ('lat'))
    lons = ds.createVariable('lon', 'f4', ('lon'))
    ghi = ds.createVariable('ghi', 'f4', ('time', 'lat', 'lon',))
    ghi.units = 'Unknown'
    
    lats[:] = latz
    lons[:] = lonz
    
    ghi[0, :, :] = ghi1
    
    ds.close()
    
    print("Part 2 is done")
    
    ## read data with xarray, extract values from nc file, and assign variables

    data = xr.open_dataset('/home/tyler/cloud_advection/operational/{}'.format(filename))

    ghi = data['ghi'].values
    ghi = np.asarray(ghi[0,:,:])

    lats = data['lat'].values
    lons = data['lon'].values
    
    
    ## calculate transform variable to create raster data

    res = (lons[-1] - lons[0]) / 240.0
    transform = Affine.translation(lons[0] - res / 2, lats[0] - res / 2) * Affine.scale(res, res)
    
    
    # open in 'write' mode, unpack profile info to dst
    ## Create raster file

    with rio.open(
       'erebos_grid_{0}-{1}-{2}_{3}.tif'.format(year,month,day,second_index),
       "w",
       driver="GTiff",         # output file type
       height=ghi.shape[0],      # shape of array
       width=ghi.shape[1],
       count=1,                # number of bands
       dtype=ghi.dtype,          # output datatype
       crs="+proj=latlong",    # CRS
       transform=transform,    # location and resolution of upper left cell
    ) as dst:
       # check for number of bands
       if dst.count == 1:
           # write single band
           dst.write(ghi, 1)
       else:
           # write each band individually
           for band in range(len(ghi)):
               # write data, band # (starting from 1)
               dst.write(ghi[band], band + 1)
            
    return "Job is Done"


In [23]:
grid_maker(central_lon=-110.95534, central_lat=32.22969, year='2021', month='07', day='11', second_index=0)

Part 1 is done
Part 2 is done


'Job is Done'

In [ ]:
for i in range(1,10):
    grid_maker(central_lon=-110.95534, central_lat=32.22969, year='2021', month='07', day='11', second_index=i)
    


Part 1 is done
Part 2 is done
Part 1 is done
Part 2 is done
Part 1 is done
